In [11]:
# Я скачал датасет mtg-jamendo moodtheme с типом audio-low и таблицу raw (она в репозитории была)
#
# Клонировал репозиторий
#
# git clone https://github.com/MTG/mtg-jamendo-dataset.git
# cd mtg-jamendo-dataset
#
# Дальше создал среду, чтобы скачать один требуемый пакет
#
# uv init
# source .venv/bin/activate
# uv add gdown
#
# Затем скачал сам датасет
#
# python3 scripts/download/download.py --dataset autotagging_moodtheme --type audio-low ../mtg-jamendo-dataset/datasets --unpack --remove
#
# Скачанный датасет 90% перенёс в папку с моим проектом в data/train и 10% в папку data/val

# Аудиофайлы в датасете были с дополнительным .low перед .mp3 в названии
# Поэтому я переименовал все файлы убрав .low, чтобы названия полностью совпадали с таблицей raw

import os

_path = "../data/mtg-jamendo/audio/"
for i in range(100):
    
    if i >= 10: path = _path + str(i)
    else: path = _path + '0' + str(i)

    for f in os.listdir(path):
        old = os.path.join(path, f)
        if os.path.isfile(old) and '.low.' in f:
            new = os.path.join(path, f.replace('.low.', '.'))
            os.rename(old, new)

In [18]:
# Теперь попробую открыть какой-нибудь аудиофайл и просмотреть информацию по нему из таблицы

import os
import librosa
import soundfile as sf
import random

def cut_random_30s(input_dir, output_dir):
    for root, dirs, files in os.walk(input_dir):
        for file in files:
            if file.endswith(('.mp3', '.wav', '.flac')):
                input_path = os.path.join(root, file)
                relative_path = os.path.relpath(root, input_dir)
                output_subdir = os.path.join(output_dir, relative_path)
                os.makedirs(output_subdir, exist_ok=True)
                
                try:
                    audio, sr = librosa.load(input_path, sr=None)
                    total_duration = len(audio) / sr
                    
                    if total_duration > 30:
                        max_start = total_duration - 30
                        start_time = random.uniform(0, max_start)
                        start_sample = int(start_time * sr)
                        end_sample = start_sample + 30 * sr
                        audio_30s = audio[start_sample:end_sample]
                    else:
                        audio_30s = audio
                    
                    output_path = os.path.join(output_subdir, file)
                    sf.write(output_path, audio_30s, sr)
                    print(f"Processed: {input_path}")
                except Exception as e:
                    print(f"Error with {input_path}: {str(e)}")

input_dir = "../data/train/audio/"
output_dir = "../data/temp/"
i = 1
cut_random_30s(input_dir + str(i).zfill(2), output_dir + str(i).zfill(2))

